# Machine Learning model for integration with Devops (MLOps)

### Model creation and training using MNIST dataset

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import np_utils
import keras
import sys

Using TensorFlow backend.
C:\Users\mangl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mangl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mangl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mangl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

### Loading mnist dataset

In [2]:
(x_train, y_train), (x_test, y_test)  = mnist.load_data()  # loads the MNIST dataset

# Store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

#### To Get right 'shape' needed for Keras we add a 4th dimension to the model by changing our original image shape of (60000,28,28) to (60000,28,28,1)

In [3]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [4]:
input_shape = (img_rows, img_cols, 1)   # store the shape of a single image 

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#### Normalize our data by changing the range from (0 to 255) to (0 to 1)

In [5]:
x_train /= 255
x_test /= 255

#### Now we one hot encode outputs

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

## Model creation

In [7]:
model = Sequential()

#### Adding Layers to model

In [8]:
# first set of CRP (Convolution, RELU, Pooling)

def input_layers(conv,nfilter1,filter_size1,pool_size1,fc_input,no_neurons):
    #conv = int(input('conv layers :'))
    #nfilter1 = int(input('filter layer :'))
    #filter_size1 = int(input('filter size :'))
    #pool_size1 = int(input('pool layer :'))
    this_layer = 'No. of convolve layers : ' + str(conv)
    this_layer = this_layer + '\nLayer 1'
    this_layer = this_layer + '\nNo of filters : ' + str(nfilter1) + '\nFilter Size : ' + str(filter_size1) + '\nPool Size : ' + str(pool_size1)

    model.add(Conv2D(nfilter1, (filter_size1,filter_size1),padding = "same",input_shape = input_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (pool_size1,pool_size1)))

    #Subsequent CRP sets
    for i in range(1,conv):
        nfilters = int(input('filters layers :'))
        filter_size = int(input('filters size :'))
        pool_size = int(input('pool size :'))
        this_layer = this_layer + '\nLayer ' + str(i+1) + ': '
        this_layer = this_layer + '\nNo of filters : ' + str(nfilters) + '\nFilter Size : ' + str(filter_size) + '\nPool Size : ' + str(pool_size)
        model.add(Conv2D(nfilters, (filter_size, filter_size),padding = "same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (pool_size, pool_size)))

    model.add(Flatten())  # Fully connected layers (w/ RELU)

    #fc_input = int(input('FC layer :'))

    this_layer = this_layer + '\nNo. of FC Layers : ' + str(fc_input+1) 

    for i in range(0,fc_input):
        #no_neurons = int(input('neurons :'))
        this_layer = this_layer + '\nNeurons in Layer ' + str(i+1) + ' : ' + str(no_neurons)
        model.add(Dense(no_neurons))
        model.add(Activation("relu"))

    # Softmax (for classification)
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    this_layer = this_layer + '\nNeurons in Layer ' + str(fc_input + 1) + ' : ' + str(num_classes)
    model.compile(loss = 'categorical_crossentropy',optimizer = keras.optimizers.Adadelta(),metrics = ['accuracy'])
    
    print(model.summary())

input_layers(1,32,3,3,0,16)    # Adding layers to layers function


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2592)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                25930     
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 26,250
Trainable params: 26,250
Non-trainable params: 0
_________________________________________________

### Now let us train LeNet on our MNIST Dataset

In [9]:
def training():
    batch_size = 128
    epochs = 5
    
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    model.save("mlops.h5")

    # Evaluate the performance of our trained model
    global scores
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    
training()


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 23s 381us/step - loss: 0.3636 - accuracy: 0.8970 - val_loss: 0.1510 - val_accuracy: 0.9576
Epoch 2/5
60000/60000 [==============================] - 22s 361us/step - loss: 0.1318 - accuracy: 0.9622 - val_loss: 0.0986 - val_accuracy: 0.9710
Epoch 3/5
60000/60000 [==============================] - 23s 380us/step - loss: 0.0970 - accuracy: 0.9718 - val_loss: 0.0808 - val_accuracy: 0.9758
Epoch 4/5
60000/60000 [==============================] - 22s 362us/step - loss: 0.0816 - accuracy: 0.9762 - val_loss: 0.0716 - val_accuracy: 0.9777
Epoch 5/5
10000/10000 [==============================] - 2s 208us/step
Test loss: 0.0685330237729475
Test accuracy: 0.979200005531311


#### Retraining the model by checking accuracy

In [ ]:
if scores[1] < 0.95 :
    input_layers(2,64,3,3,1,32)
    training()

 ### Save accuracy to particular file

In [ ]:
accuracy_file = open('/MLops/accuracy.txt','w') 
accuracy_file.write(str(scores[1]))
accuracy_file.close()